# Plot NTU joint trajectory

Configure a pose file, joint index, and frame range below to plot the X/Y/Z trajectories for a single clip.

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# --- configuration ---
pose_path = Path("../datasets/ntu/p001/S001C001P001R001A001_pose_50hz.npy")
joint_idx = 4  # 0-based (e.g., 4=left shoulder, 5=left elbow, 6=left wrist, 8/9/10 right arm)
start_frame = 0
end_frame = None  # set to an int to clip the range


def load_pose(path: Path) -> np.ndarray:
    arr = np.load(path, mmap_mode="r")
    if arr.ndim != 3:
        raise ValueError(f"Expected 3D pose tensor, got shape {arr.shape}")
    pose = arr
    if pose.shape[0] == 3 and pose.shape[2] < pose.shape[1]:
        pose = pose.transpose(0, 2, 1)  # (3, frames, joints) -> (3, joints, frames)
    elif pose.shape[-1] == 3:
        pose = pose.transpose(2, 1, 0)  # (frames, joints, 3) -> (3, joints, frames)
    return pose


pose = load_pose(pose_path)
num_frames = pose.shape[2]
s = max(0, start_frame)
e = min(num_frames, end_frame) if end_frame is not None else num_frames

+    coords = pose[:, joint_idx, s:e]
frames = np.arange(s, e)

fig, ax = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels = ["x", "y", "z"]
for i in range(3):
        ax[i].plot(frames, coords[i], label=labels[i])
        ax[i].set_ylabel(labels[i])
        ax[i].legend()
ax[-1].set_xlabel("frame")
fig.suptitle(f"Joint {joint_idx} | frames {s}–{e-1} | {pose_path.name}")
plt.tight_layout()
plt.show()